# Data from Analise's Thesis

In [1]:
import os
os.chdir('..')

In [2]:
import pandas as pd
from src.plot_utils import PlotUtils
from src.analise_thesis.plotter import Plotter
import matplotlib.pyplot as plt
from pprint import pprint

### Set general figure parameters

In [7]:
home = os.path.expanduser("~")
figure_store_path = os.path.join(home_dir, 'data/figs_sensors_actuators')
os.makedirs(figure_store_path, mode = 0o777, exist_ok = True)
fig_format = 'svg'
tick_fontsize = 14
PlotUtils.set_user_figure_dir(user_fig_directory=figure_store_path)

## Hard chip data

In [4]:
data_hard = [
    (896, 1, 'hard'),
    (764, 1, 'hard'),
    (608, 1, 'hard'),
    (512, 1, 'hard'),
    (764, 2, 'hard'),
    (608, 2, 'hard'),
    (512, 2, 'hard'),
    (384, 2, 'hard'),
    (288, 2, 'hard'),
    (996, 3, 'hard'),
    (800, 3, 'hard'),
    (764, 3, 'hard'),
    (764, 4, 'hard'),
    (512, 4, 'hard'),
    (384, 4, 'hard'),
    (192, 4, 'hard'),
    ]
plotter_hard = Plotter(data = data_hard, nrows = 1, ncols = 1,
                       xlabel = "injection number", ylabel = 'R [K$\Omega$]',
                       xlim = [0, 5], ylim = [0, 1e6], capsize = 10,
                       fig_format = fig_format, tick_fontsize=tick_fontsize)

In [5]:
plotter_hard.sort_tuples(data=data_hard, idx_to_sort_by=0)
pprint(data_hard)
# special = filter_by_width(data, 764)
# print(special)

[(192, 4, 'hard'),
 (288, 2, 'hard'),
 (384, 2, 'hard'),
 (384, 4, 'hard'),
 (512, 1, 'hard'),
 (512, 2, 'hard'),
 (512, 4, 'hard'),
 (608, 1, 'hard'),
 (608, 2, 'hard'),
 (764, 1, 'hard'),
 (764, 2, 'hard'),
 (764, 3, 'hard'),
 (764, 4, 'hard'),
 (800, 3, 'hard'),
 (896, 1, 'hard'),
 (996, 3, 'hard')]


### Create plots for each data tuple for all hard data

In [6]:
plotter_hard.run_individual_chips()

[Errno 2] No such file or directory: '/home/mkrause/data/artificial-muscle/analise_thesis/CNF Data.xlsx': check if that Excel file exists in the directory - if not add it


SystemExit: Can't find Excel file /home/mkrause/data/artificial-muscle/analise_thesis/CNF Data.xlsx - aborting

/home/mkrause/dev/artificial-muscle/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Create a Pandas Dataframe for all _hard_ data

In [ ]:
df = pd.DataFrame()
channel_widths = plotter_hard.get_channel_list()
for channel_width in channel_widths:
    data = plotter_hard.filter_by_width(width=channel_width)
    df_ = pd.DataFrame(plotter_hard.get_averaged_channel_data(data))
    df = pd.concat([df_, df])

### Print dataframe and write to disk as a CSV file

In [ ]:
path = os.path.join(PlotUtils.glob_figpath, 'weighted_data_hard.csv')
df.to_csv(path)
df

### Filter dataframe for the resistance at the last injection

In [ ]:
df_last_injection = df.loc[df.x_data == df.number_of_injections]
df_last_injection

### Summary plots: resistance at each channel after last injection
The number of the last injection is variable, shown in the above tables in column `number_of_injections`.

In [ ]:
# print(df_last_injection.chip_width.to_list())
title = 'Channel resistance after the last injection (hard)'
xlabel = 'Channel width ($\mu m$)'
ylabel = 'Resistance (k$\Omega$)'
# with linear y-scale
figname = os.path.join(PlotUtils.glob_figpath, f'resistance_at_last_injection_hard_linear.{fig_format}')
PlotUtils.plot_scatter(df_last_injection.chip_width, df_last_injection.y_data, yerr=df_last_injection.y_err,
                       nrows=plotter_hard.nrows, ncols=plotter_hard.ncols, 
                       xlabel=plotter_hard.xlabel, ylabel=plotter_hard.ylabel, capsize=plotter_hard.capsize,
                       xticks=df_last_injection.chip_width.to_list(), title=title, figname=figname,
                       fig_format=fig_format)
# with logarithmic y-scale
figname = os.path.join(PlotUtils.glob_figpath, f'resistance_at_last_injection_hard_logarithmic.{fig_format}')
PlotUtils.plot_scatter(df_last_injection.chip_width, df_last_injection.y_data, yerr=df_last_injection.y_err,
                       nrows=plotter_hard.nrows, ncols=plotter_hard.ncols, 
                       xlabel=plotter_hard.xlabel, ylabel=plotter_hard.ylabel, capsize=plotter_hard.capsize,
                       xticks=df_last_injection.chip_width.to_list(), title=title, figname=figname,
                       fig_format=fig_format, yscale='log')

## Soft chip data

In [ ]:
data_soft = [
    (720, 1, 'soft'),
    (720, 3, 'soft'),
    (720, 5, 'soft'),
    (720, 6, 'soft'),
    (960, 1, 'soft'),
    (960, 5, 'soft'),
    (840, 1, 'soft'),
    (840, 5, 'soft'),
    (840, 6, 'soft'),
    (1080, 1, 'soft'),
    (1080, 2, 'soft'),
    (1080, 3, 'soft'),
    (1080, 5, 'soft'),
    (1080, 6, 'soft'),
    (1080, 7, 'soft'),
    (600, 2, 'soft'),
    (600, 3, 'soft'),
    (600, 5, 'soft'),
    (480, 2, 'soft'),
]
plotter_soft = Plotter(data = data_soft, nrows = 1, ncols = 1, 
                       xlabel = "injection number", ylabel = 'R [K$\Omega$]',
                       xlim = [0, 5], ylim = [0, 1e6], capsize = 10,
                       fig_format = fig_format)

### Create plots for each data tuple

In [ ]:
plotter_soft.run_individual_chips()

In [ ]:
df = pd.DataFrame()
channel_widths = plotter_soft.get_channel_list()
for channel_width in channel_widths:
    data = plotter_soft.filter_by_width(width=channel_width)
    df_ = pd.DataFrame(plotter_soft.get_averaged_channel_data(data))
    df = pd.concat([df_, df])

In [ ]:
path = os.path.join(PlotUtils.glob_figpath, 'weighted_data_soft.csv')
df.to_csv(path)
df

### Filter dataframe for last injection

In [ ]:
df_last_injection = df.loc[df.x_data == df.number_of_injections]
df_last_injection

### Summary plots: resistance at each channel after last injection
The number of the last injection is variable, shown in the above tables in column `number_of_injections`.

In [ ]:
print(df_last_injection.chip_width.to_list())
title = 'Channel resistance after the last injection (soft)'
xlabel = 'Channel width ($\mu m$)'
ylabel = 'Resistance (k$\Omega$)'

# with linear y-scale
figname = os.path.join(PlotUtils.glob_figpath, f'resistance_at_last_injection_soft_linear.{fig_format}')
PlotUtils.plot_scatter(df_last_injection.chip_width, df_last_injection.y_data, yerr=df_last_injection.y_err, 
                       nrows=plotter_soft.nrows, ncols=plotter_soft.ncols, 
                       xlabel=plotter_soft.xlabel, ylabel=plotter_soft.ylabel, capsize=plotter_soft.capsize,
                       xticks=df_last_injection.chip_width.to_list(), title=title, figname=figname,
                       fig_format=fig_format, ylim=[0, 200])

# with logrithmic y-scale
figname = os.path.join(PlotUtils.glob_figpath, f'resistance_at_last_injection_soft_logarithmic.{fig_format}')
PlotUtils.plot_scatter(df_last_injection.chip_width, df_last_injection.y_data, yerr=df_last_injection.y_err, 
                       nrows=plotter_soft.nrows, ncols=plotter_soft.ncols, 
                       xlabel=plotter_soft.xlabel, ylabel=plotter_soft.ylabel, capsize=plotter_soft.capsize,
                       xticks=df_last_injection.chip_width.to_list(), title=title, figname=figname,
                       fig_format=fig_format, ylim=[0, 200], yscale='log')